# Retreiving and Preparing Text for Machines

In [11]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud #pip install git+git://github.com/Computational-Content-Analysis-2018/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib 
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

from IPython.display import display

### Scraping Text (from saved .html files)

In [12]:
countryDict = {'Saudi Arabia': [
"saud1.html",
"saud2.html",
"saud3.html",
"saud4.html",
"saud5.html",
"saud6.html",
"saud7.html",
"saud8.html",
"saud9.html",
"saud10.html",
"saud11.html",
"saud12.html",
"saud13.html",
"saud14.html",
"saud15.html"] ,
               'China': [
"china1.html",
"china2.html",
"china3.html",
"china4.html",
"china5.html",
"china6.html",
"china7.html",
"china8.html",
"china9.html",
"china10.html",
"china11.html",
"china12.html",
"china13.html",
"china14.html",
"china15.html"],
               'US': [
"us01.html",
"us02.html",
"us03.html",
"us04.html",
"us05.html",
"us06.html",
"us07.html",
"us08.html",
"us09.html",
"us10.html",
"us11.html",
"us12.html",
"us13.html",
"us14.html"],
               'UK': [
"uk01.html",
"uk02.html",
"uk03.html",
"uk04.html",
"uk05.html",
"uk06.html",
"uk07.html",
"uk08.html",
"uk09.html",
"uk10.html",
"uk11.html",
"uk12.html",
"uk13.html",
"uk14.html"],
               'Australia': [
"au01.html",
"au02.html",
"au03.html",
"au04.html",
"au05.html",
"au06.html",
"au07.html",
"au08.html",
"au09.html",
"au10.html",
"au11.html",
"au12.html",
"au13.html",
"au14.html"],
               'Kenya': [
"kenya1.html",
"kenya2.html",
"kenya3.html",
"kenya4.html",
"kenya5.html",
"kenya6.html",
"kenya7.html",
"kenya8.html",
"kenya9.html",
"kenya10.html",
"kenya11.html",
"kenya12.html",
"kenya13.html",
"kenya14.html",
"kenya15.html"],
               'India': [
"india1.html",
"india2.html",
"india3.html",
"india4.html",
"india5.html",
"india6.html",
"india7.html",
"india8.html",
"india9.html",
"india10.html",
"india11.html",
"india12.html",
"india13.html",
"india14.html",
"india15.html"]}              

In [21]:
def loopThruArticles(countryDict):

    parsDict = {'region': [], 'source': [], 'title': [], 'text': [], 'date': [], 'author': [], 'section': []}

    for key, val in countryDict.items():
        for html in val:
            parsDict['region'].append(key)
            parseArticle(html, parsDict)
    
    return parsDict
    

In [24]:
def parseArticle(html, parsDict):
    soup = bs4.BeautifulSoup(open(html, 'rt', encoding='UTF8'),'lxml')

    # title
    title0 = soup.find_all('div', {'class' : 'title'})
    title1 = title0[0].text.replace('Hide Details', '')
    title = re.sub(r'\<.*\>', '', title1)
    parsDict['title'].append(title)

    # text
    body = soup.find_all('div', {'class' : 'body'})
    text0 = body[0].text
    text1 = re.sub(r'\n', ' ', text0) #Take out new lines
    text2 = re.sub(r"Source- .+","", text1) #for Times of India
    text = re.sub(r"© .+","", text2) #for allAfrica
    parsDict['text'].append(text)
    
    with open("%s.txt" % html[:-5], "w", encoding='UTF-8') as text_file:
        text_file.write(text)

    # source & date
    source = soup.find_all('div', {'class' : 'source'})
    date = re.findall(r'(?:January|February|March|April|May|June|July|August|September|October|November|Dececember)\s\d{1,2},\s\d{4}', source[0].text)
    parsDict['date'].append(date)
    pat2 = re.compile(r"(.*?)-", re.M)
    newssource = pat2.findall(source[0].text)
    parsDict['source'].append(newssource[0])

    meta = soup.find_all('span', {'class' : 'val'})
    # author    
    author0 = meta[0]
    author = re.sub(r'\<.*\>', '', author0.text) #Take out <>
    parsDict['author'].append(author)

    # section
    section0 = meta[1]
    section = re.sub(r'\<.*\>', '', section0.text) #Take out <>
    parsDict['section'].append(section)
    

In [25]:
df = pandas.DataFrame(loopThruArticles(countryDict))

In [19]:
df.to_csv('news.csv', index_label = False)